In [20]:
import os
import csv
from supabase import create_client, Client
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [38]:
def update_game_schedule(game_schedule_csv):
    game_insert = []
    with open(game_schedule_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            print(row)
            if row != {'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}:
                game_insert.append(row)

    data = supabase.table("game").upsert(game_insert,
                                         # ignore_duplicates=True, on_conflict="game_date, team_1_id, team_2_id"
                                         ).execute()
    print(data)

In [39]:
round = "3"
update_game_schedule(f"game-schedule-round-{round}-2023-ncaa-tournament.csv")

{'game_date': '2023-03-18', 'team_2_id': 'san-diego-state', 'team_1_id': 'furman', 'game_time': '12:30', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'tennessee', 'team_1_id': 'duke', 'game_time': '14:40', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'kansas', 'team_1_id': 'arkansas', 'game_time': '17:15', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'missouri', 'team_1_id': 'princeton', 'game_time': '18:10', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'houston', 'team_1_id': 'auburn', 'game_time': '19:10', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'texas', 'team_1_id': 'penn-state', 'game_time': '19:45', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'ucla', 'team_1_id': 'northwestern', 'game_time': '20:40', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2

In [25]:
import pandas as pd
import datetime
def generate_game_scoring_sheet(competition_id: int = 1, date = datetime.date.today()):
    data = supabase.table("players_in_games_view").select("*").eq("game_date", date).eq('competition_id', competition_id).execute()
    df = pd.DataFrame(data.data)
    df['points'] = None
    # df['rebounds'] = 0
    # df['assists'] = 0
    df.to_csv(f"{date}-game-scoring-2023-ncaa-tournament.csv", index=False, columns=['game_time', 'team_unique', 'player_unique', 'points','game_id'])
    # 'rebounds', 'assists'])

In [40]:
date = datetime.date.today()
generate_game_scoring_sheet(date=date)

In [15]:
def update_scores_from_csv(current_round: int, competition_id: int = 1, game_scoring_csv = f"{datetime.date.today()}-game-scoring-2023-ncaa-tournament.csv"):
    player_game_insert = []
    with open(game_scoring_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        next(reader)
        for row in reader:
            if row['points'] != '':
                player_game_row = {
                    "player_unique": row['player_unique'],
                    "game_id": row['game_id'],
                    "points": row['points'],
                }
                player_game_insert.append(player_game_row)

    data = supabase.table("player_game").upsert(player_game_insert,
                                         ignore_duplicates=False, on_conflict="game_id, player_unique").execute()
    updated = supabase.table("competition_updated").insert({"competition_id": competition_id, "current_round": current_round}).execute()
    print(data)

In [41]:
date = datetime.date.today()
# date = "2023-03-16"
update_scores_from_csv(current_round=2, game_scoring_csv = f"{date}-game-scoring-2023-ncaa-tournament.csv")

data=[{'player_unique': 'jalen-slawson-1', 'game_id': 64, 'points': 8, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'jp-pegues-1', 'game_id': 64, 'points': 10, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'marcus-foster-2', 'game_id': 64, 'points': 5, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'garrett-hien-1', 'game_id': 64, 'points': 3, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'ben-vanderwal-1', 'game_id': 64, 'points': 0, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'alex-williams-3', 'game_id': 64, 'points': 11, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'tyrese-hughey-1', 'game_id': 64, 'points': 0, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'carter-whitt-1', 'game_id': 64, 'points': 0, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'joe-anderson-4', 'game_id': 64, 'points': 0, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'jonny-la